In [58]:
#| output: false

from __future__ import annotations

import os
import torch
os.environ["KERAS_BACKEND"] = "torch"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import keras
from keras import layers

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from datetime import timedelta
from dateutil.relativedelta import relativedelta
from fastcore import docments
from nbdev.showdoc import show_doc
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [59]:
df_weekly = pd.read_csv("..\\data\\weekly.csv")
df_monthly = pd.read_csv("..\\data\\monthly.csv")

In [60]:
unnamedCols = [col for col in df_weekly.columns if 'Unnamed' in col]
df_weekly.drop(columns=unnamedCols, inplace=True)
unnamedCols = [col for col in df_monthly.columns if 'Unnamed' in col]
df_monthly.drop(columns=unnamedCols, inplace=True)

In [61]:
df_weekly

,INSTITUTIONCODE,REPORTINGDATE,DESC_NO,ASSET_AMOUNT,LIABILITY_AMT
0,B5114,15.01.10,17.0,0.000000e+00,0.000000e+00
1,B5114,15.01.10,18.0,2.786249e+09,0.000000e+00
2,B5114,15.01.10,19.0,2.511281e+09,0.000000e+00
3,B5114,15.01.10,20.0,2.649853e+08,0.000000e+00
4,B5114,15.01.10,21.0,0.000000e+00,0.000000e+00
...,...,...,...,...,...
821704,Unsampled,01.01.16,13.0,2.418254e+08,0.000000e+00
821705,B5812,01.01.16,12.0,1.059575e+10,1.059575e+10
821706,B5812,01.01.16,11.0,2.133258e+10,2.133258e+10
821707,B5812,01.01.16,10.0,1.490400e+11,1.490400e+11


In [62]:
df_monthly

,INSTITUTIONCODE,BSH_REPORTINGDATE,F001_ASSET_CASH,F002_ASSET_BAL_BOT,F003_ASSET_BAL_BOT_SMR,F004_ASSET_BAL_BOT_CURRENT_ACCOUNT,F005_ASSET_BAL_BOT_OTHERS,F006_ASSET_BAL_OTHER_BANKS,F007_ASSET_BAL_OTHER_BANKS_TZ,F008_ASSET_BAL_OTHER_BANKS_ABROAD,...,EWAQ_Capital,EWAQ_NPL,EWAQ_GrossLoans,EWAQ_LargeExposures,EWAQ_NPLsNetOfProvisions,EWAQ_NPLsNetOfProvisions2CoreCapital,EWAQ_NPLs2GrossLoans,EWAQ_AssetsQualityRating,EWAQ_Loans,MLA_CLASS
0,B5412,31.01.10,9.570422e+09,5.260077e+10,3.700000e+10,1.560077e+10,0.0,1.350000e+11,1.622160e+10,1.190000e+11,...,9.514540e+10,1.951219e+09,1.270000e+11,7.137633e+10,903220975.0,0.0095,0.0153,1.0,NaN,high
1,B5412,28.02.10,6.996645e+09,3.554718e+10,3.200000e+10,3.547185e+09,0.0,1.480000e+11,7.535208e+08,1.470000e+11,...,9.595342e+10,1.954516e+09,1.250000e+11,7.831967e+10,904869349.0,0.0094,0.0156,1.0,NaN,high
2,B5412,31.03.10,8.689629e+09,3.744132e+10,3.350000e+10,3.941316e+09,0.0,1.620000e+11,5.570919e+09,1.570000e+11,...,9.864602e+10,1.979268e+09,1.280000e+11,7.846886e+10,917245663.0,0.0093,0.0154,1.0,NaN,high
3,B5412,30.04.10,6.811311e+09,3.527009e+10,3.250000e+10,2.770089e+09,0.0,2.240000e+11,8.739334e+09,2.160000e+11,...,9.960755e+10,2.014398e+09,1.290000e+11,8.986596e+10,934810301.0,0.0094,0.0157,1.0,NaN,high
4,B5412,31.05.10,9.160128e+09,4.927299e+10,3.982500e+10,9.447986e+09,0.0,1.890000e+11,1.524899e+09,1.880000e+11,...,1.010000e+11,2.081954e+09,1.320000e+11,8.567217e+10,968588451.0,0.0096,0.0158,1.0,NaN,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4688,B5120,31.08.21,6.934755e+08,1.703142e+09,0.000000e+00,1.703142e+09,0.0,2.742275e+08,2.746237e+08,0.000000e+00,...,9.314589e+10,0.000000e+00,2.274679e+10,1.653192e+10,-5807292.0,-0.0001,0.0000,1.0,NaN,high
4689,B5120,30.09.21,3.887219e+08,1.388285e+09,0.000000e+00,1.388285e+09,0.0,8.007882e+08,8.035486e+08,0.000000e+00,...,9.337502e+10,0.000000e+00,2.301912e+10,1.657398e+10,-5749760.0,-0.0001,0.0000,1.0,NaN,high
4690,B5120,31.10.21,4.772036e+08,7.915250e+08,0.000000e+00,7.915250e+08,0.0,2.992667e+08,3.003208e+08,0.000000e+00,...,9.359472e+10,0.000000e+00,2.318983e+10,3.300312e+10,-3640727.0,0.0000,0.0000,1.0,NaN,high
4691,B5120,30.11.21,5.124198e+08,8.769145e+08,0.000000e+00,8.769145e+08,0.0,9.021986e+08,9.052311e+08,0.000000e+00,...,9.378179e+10,0.000000e+00,2.288809e+10,1.617025e+10,-3554388.0,0.0000,0.0000,1.0,NaN,high


In [63]:
mask1 = (df_weekly['DESC_NO'] <= 12)
df_weekly['AMOUNT']=np.where(mask1,df_weekly['LIABILITY_AMT'],df_weekly['ASSET_AMOUNT'])
dfX_01 = df_weekly.pivot_table(index=['INSTITUTIONCODE','REPORTINGDATE'], columns=['DESC_NO'], values='AMOUNT')
dfX_01 

DESC_NO                                1.0           2.0           3.0   \
INSTITUTIONCODE REPORTINGDATE                                             
B5014           01.01.16       7.398879e+10  5.628409e+10  9.143355e+10   
                01.01.21       1.795270e+11  6.045049e+10  1.849990e+11   
                01.02.13       2.806429e+10  2.871621e+10  4.605558e+10   
                01.02.19       1.803850e+11  5.977419e+10  1.504990e+11   
                01.03.13       6.784919e+10  2.463718e+10  4.675235e+10   
...                                     ...           ...           ...   
Unsampled       31.08.12       2.701363e+09  1.258243e+10  5.267539e+09   
                31.08.18       4.629424e+09  2.553700e+10  5.334408e+09   
                31.10.14       5.044337e+09  1.529065e+10  7.322940e+09   
                31.12.10       6.763083e+09  3.532561e+10  2.837152e+09   
                31.12.21       8.229725e+09  2.544589e+10  6.746725e+09   

DESC_NO                                4.0           5.0           6.0   \
INSTITUTIONCODE REPORTINGDATE                                             
B5014           01.01.16       2.559228e+08  0.000000e+00  0.000000e+00   
                01.01.21       4.886609e+08  4.961363e+08  0.000000e+00   
                01.02.13       6.198604e+08  0.000000e+00  0.000000e+00   
                01.02.19       1.168116e+09  1.034751e+08  0.000000e+00   
                01.03.13       6.241014e+08  0.000000e+00  0.000000e+00   
...                                     ...           ...           ...   
Unsampled       31.08.12       1.230827e+09  3.191502e+09  4.245538e+07   
                31.08.18       1.142667e+09  2.416068e+09  2.230202e+07   
                31.10.14       1.910804e+09  5.231054e+09  2.308233e+07   
                31.12.10       3.750779e+09  1.038268e+10  0.000000e+00   
                31.12.21       2.541596e+09  1.211769e+10  9.975789e+08   

DESC_NO                               7.0           8.0           9.0   \
INSTITUTIONCODE REPORTINGDATE                                            
B5014           01.01.16               0.0  4.029011e+08  2.570451e+09   
                01.01.21               0.0  3.061811e+08  0.000000e+00   
                01.02.13               0.0  2.773075e+08  3.300202e+09   
                01.02.19               0.0  4.193475e+08  3.272732e+10   
                01.03.13               0.0  3.082848e+08  3.295242e+09   
...                                    ...           ...           ...   
Unsampled       31.08.12               0.0  6.176834e+06  1.104298e+07   
                31.08.18               0.0  6.030663e+06  1.604193e+08   
                31.10.14               0.0  3.032088e+06  1.186188e+07   
                31.12.10               0.0  1.132299e+08  0.000000e+00   
                31.12.21       250000000.0  2.405258e+06  4.273882e+07   

DESC_NO                                10.0  ...          17.0          18.0  \
INSTITUTIONCODE REPORTINGDATE                ...                               
B5014           01.01.16       3.199441e+08  ...  0.000000e+00  1.406108e+10   
                01.01.21       1.431990e+11  ...  0.000000e+00  2.015696e+10   
                01.02.13       8.859703e+08  ...  0.000000e+00  1.586030e+09   
                01.02.19       1.352940e+11  ...  5.635403e+09  7.599516e+09   
                01.03.13       6.371758e+08  ...  0.000000e+00  7.710430e+08   
...                                     ...  ...           ...           ...   
Unsampled       31.08.12       1.127494e+10  ...  0.000000e+00  3.075694e+09   
                31.08.18       2.612991e+10  ...  0.000000e+00  9.280298e+09   
                31.10.14       1.680082e+10  ...  5.295485e+06  5.953266e+09   
                31.12.10       3.417645e+10  ...  0.000000e+00  1.379408e+10   
                31.12.21       1.039779e+10  ...  1.600121e+07  1.639548e+10   

DESC_NO                                19.0 

In [64]:
dfX_01.rename(columns={1: "01_CURR_ACC", 
                    2: "02_TIME_DEPOSIT",
                    3: "03_SAVINGS",
                    4: "04_OTHER_DEPOSITS",
                    5: "05_BANKS_DEPOSITS",
                    6: "06_BORROWING_FROM_PUBLIC",
                    7: "07_INTERBANKS_LOAN_PAYABLE",
                    8: "08_CHEQUES_ISSUED",
                    9: "09_PAY_ORDERS",
                    10:"10_FOREIGN_DEPOSITS_AND_BORROWINGS",
                    11:"11_OFF_BALSHEET_COMMITMENTS",
                    12:"12_OTHER_LIABILITIES",
                    13:"13_CASH",
                    14:"14_CURRENT_ACC",
                    15:"15_SMR_ACC",
                    16:"16_FOREIGN CURRENCY",
                    17:"17_OTHER DEPOSITS",
                    18:"18_BANKS_TZ",
                    19:"19_BANKS_ABROAD",
                    20:"20_CHEQUES_ITEMS_FOR_CLEARING",
                    21:"21_INTERBANK_LOANS",
                    22:"22_TREASURY_BILLS",
                    23:"23_OTHER_GOV_SECURITIES",
                    24:"24_FOREIGN_CURRENCY",
                    25:"25_COMMERCIAL_BILLS",
                    26:"26_PROMISSORY_NOTES"
                    },inplace=True)

In [65]:
dfX_01=dfX_01.reset_index()

In [66]:
dfX_01=dfX_01[(dfX_01['INSTITUTIONCODE']!='Unsampled')&(dfX_01['INSTITUTIONCODE']!=None)]

In [67]:
dfX_01

DESC_NO,INSTITUTIONCODE,REPORTINGDATE,01_CURR_ACC,02_TIME_DEPOSIT,03_SAVINGS,04_OTHER_DEPOSITS,05_BANKS_DEPOSITS,06_BORROWING_FROM_PUBLIC,07_INTERBANKS_LOAN_PAYABLE,08_CHEQUES_ISSUED,...,17_OTHER DEPOSITS,18_BANKS_TZ,19_BANKS_ABROAD,20_CHEQUES_ITEMS_FOR_CLEARING,21_INTERBANK_LOANS,22_TREASURY_BILLS,23_OTHER_GOV_SECURITIES,24_FOREIGN_CURRENCY,25_COMMERCIAL_BILLS,26_PROMISSORY_NOTES
0,B5014,01.01.16,7.398879e+10,5.628409e+10,9.143355e+10,2.559228e+08,0.000000e+00,0.0,0.0,402901067.9,...,0.000000e+00,1.406108e+10,3.927841e+10,-1.727918e+07,2.026688e+10,5.000000e+09,0.000000e+00,2.878479e+10,0.04,0.0
1,B5014,01.01.21,1.795270e+11,6.045049e+10,1.849990e+11,4.886609e+08,4.961363e+08,0.0,0.0,306181140.0,...,0.000000e+00,2.015696e+10,1.692888e+10,4.315962e+07,2.555839e+10,2.550000e+10,1.600000e+10,1.963564e+10,0.00,0.0
2,B5014,01.02.13,2.806429e+10,2.871621e+10,4.605558e+10,6.198604e+08,0.000000e+00,0.0,0.0,277307475.2,...,0.000000e+00,1.586030e+09,1.331957e+10,2.049437e+05,1.137857e+10,3.129665e+10,0.000000e+00,1.079983e+10,-0.01,0.0
3,B5014,01.02.19,1.803850e+11,5.977419e+10,1.504990e+11,1.168116e+09,1.034751e+08,0.0,0.0,419347453.8,...,5.635403e+09,7.599516e+09,5.668946e+08,5.450135e+09,1.130000e+11,2.850000e+10,2.507243e+10,0.000000e+00,0.06,0.0
4,B5014,01.03.13,6.784919e+10,2.463718e+10,4.675235e+10,6.241014e+08,0.000000e+00,0.0,0.0,308284811.6,...,0.000000e+00,7.710430e+08,1.866910e+10,1.958314e+05,5.681760e+09,2.557805e+10,0.000000e+00,1.396014e+10,-0.01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20342,B5919,31.01.20,3.580831e+09,1.423163e+09,3.291465e+08,0.000000e+00,1.610708e+09,0.0,0.0,0.0,...,0.000000e+00,3.851020e+04,4.205411e+07,6.297477e+06,1.066082e+10,2.600000e+09,0.000000e+00,0.000000e+00,0.00,0.0
20343,B5919,31.05.19,9.871527e+09,1.698387e+09,3.641791e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,5.005318e+08,8.102080e+09,0.000000e+00,0.000000e+00,5.735140e+09,0.000000e+00,1.350651e+08,0.00,0.0
20344,B5919,31.07.20,4.594674e+09,1.647843e+09,4.975718e+08,2.130367e+07,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,3.862709e+04,7.786106e+08,1.512300e+07,5.951239e+09,4.913950e+09,0.000000e+00,0.000000e+00,0.00,0.0
20345,B5919,31.08.18,6.296507e+09,1.425508e+08,1.850541e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2.487080e+09,5.956955e+09,0.000000e+00,0.000000e+00,4.433260e+09,0.000000e+00,2.274267e+07,0.00,0.0


In [68]:
#1.6 transformed dataset to include computed variables with XX prefix
#transform
def totalDeposits(f01, f02, f03, f04):
  return f01 + f02 + f03 + f04
def totalLiquidLiab(f01,f05, f06, f07, f08, f09, f10, f11, f12):
  return f01 + f05 + f06 + f07 + f08 + f09 + f10 + f11 + f12
def botBalance(f14, f15,f16,f17):
  return f14 + f15 + f16 + f17
def balOtherBanks(f18,f19):
  return f18+f19
def totalLiqAssets(f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26):
  return f13 + f14 + f15 + f16 + f17 + f18 + f19 + f20 + f21 + f22 + f23 + f24 + f25 + f26

dfX_01['XX_CUSTOMER_DEPOSITS'] =dfX_01.apply(lambda x: totalDeposits(x['01_CURR_ACC'],x['02_TIME_DEPOSIT'],x['03_SAVINGS'],x['04_OTHER_DEPOSITS']), axis=1 )
dfX_01['XX_TOTAL_LIQUID_LIAB']=dfX_01.apply(lambda x: totalLiquidLiab(x['XX_CUSTOMER_DEPOSITS'],
                                                                         x['05_BANKS_DEPOSITS'],
                                                                         x['06_BORROWING_FROM_PUBLIC'],
                                                                         x['07_INTERBANKS_LOAN_PAYABLE'],
                                                                         x['08_CHEQUES_ISSUED'],
                                                                         x['09_PAY_ORDERS'],
                                                                         x['10_FOREIGN_DEPOSITS_AND_BORROWINGS'],
                                                                         x['11_OFF_BALSHEET_COMMITMENTS'],
                                                                         x['12_OTHER_LIABILITIES']), axis=1 )
dfX_01['XX_BOT_BALANCE']=dfX_01.apply(lambda x: botBalance(x['14_CURRENT_ACC'], x['15_SMR_ACC'], x['16_FOREIGN CURRENCY'], x['17_OTHER DEPOSITS']),axis=1)
dfX_01['XX_BAL_IN_OTHER_BANKS']=dfX_01.apply(lambda x: balOtherBanks(x['18_BANKS_TZ'],x['19_BANKS_ABROAD']),axis =1)
dfX_01['XX_TOTAL_LIQUID_ASSET']=dfX_01.apply(lambda x: totalLiqAssets(x["13_CASH"],
                                                                      x["14_CURRENT_ACC"],
                                                                      x["15_SMR_ACC"],
                                                                      x["16_FOREIGN CURRENCY"],
                                                                      x["17_OTHER DEPOSITS"],
                                                                      x["18_BANKS_TZ"],
                                                                      x["19_BANKS_ABROAD"],
                                                                      x["20_CHEQUES_ITEMS_FOR_CLEARING"],
                                                                      x["21_INTERBANK_LOANS"],
                                                                      x["22_TREASURY_BILLS"],
                                                                      x["23_OTHER_GOV_SECURITIES"],
                                                                      x["24_FOREIGN_CURRENCY"],
                                                                      x["25_COMMERCIAL_BILLS"],
                                                                      x["26_PROMISSORY_NOTES"]), axis=1)
dfX_01['XX_MLA'] = dfX_01.apply(lambda x: (100 * x['XX_TOTAL_LIQUID_ASSET']/x['XX_TOTAL_LIQUID_LIAB']), axis = 1)



In [69]:
dfX_01

DESC_NO,INSTITUTIONCODE,REPORTINGDATE,01_CURR_ACC,02_TIME_DEPOSIT,03_SAVINGS,04_OTHER_DEPOSITS,05_BANKS_DEPOSITS,06_BORROWING_FROM_PUBLIC,07_INTERBANKS_LOAN_PAYABLE,08_CHEQUES_ISSUED,...,23_OTHER_GOV_SECURITIES,24_FOREIGN_CURRENCY,25_COMMERCIAL_BILLS,26_PROMISSORY_NOTES,XX_CUSTOMER_DEPOSITS,XX_TOTAL_LIQUID_LIAB,XX_BOT_BALANCE,XX_BAL_IN_OTHER_BANKS,XX_TOTAL_LIQUID_ASSET,XX_MLA
0,B5014,01.01.16,7.398879e+10,5.628409e+10,9.143355e+10,2.559228e+08,0.000000e+00,0.0,0.0,402901067.9,...,0.000000e+00,2.878479e+10,0.04,0.0,2.219623e+11,2.468379e+11,7.136215e+10,5.333950e+10,1.981265e+11,80.265835
1,B5014,01.01.21,1.795270e+11,6.045049e+10,1.849990e+11,4.886609e+08,4.961363e+08,0.0,0.0,306181140.0,...,1.600000e+10,1.963564e+10,0.00,0.0,4.254652e+11,6.014293e+11,5.159548e+10,3.708585e+10,1.891126e+11,31.443855
2,B5014,01.02.13,2.806429e+10,2.871621e+10,4.605558e+10,6.198604e+08,0.000000e+00,0.0,0.0,277307475.2,...,0.000000e+00,1.079983e+10,-0.01,0.0,1.034559e+11,1.168572e+11,2.457452e+10,1.490560e+10,1.071388e+11,91.683533
3,B5014,01.02.19,1.803850e+11,5.977419e+10,1.504990e+11,1.168116e+09,1.034751e+08,0.0,0.0,419347453.8,...,2.507243e+10,0.000000e+00,0.06,0.0,3.918263e+11,5.729315e+11,6.779256e+10,8.166411e+09,2.647954e+11,46.217626
4,B5014,01.03.13,6.784919e+10,2.463718e+10,4.675235e+10,6.241014e+08,0.000000e+00,0.0,0.0,308284811.6,...,0.000000e+00,1.396014e+10,-0.01,0.0,1.398628e+11,1.530413e+11,2.630510e+10,1.944015e+10,1.135819e+11,74.216474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20342,B5919,31.01.20,3.580831e+09,1.423163e+09,3.291465e+08,0.000000e+00,1.610708e+09,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.00,0.0,5.333141e+09,2.696433e+10,2.131524e+09,4.209262e+07,1.570771e+10,58.253662
20343,B5919,31.05.19,9.871527e+09,1.698387e+09,3.641791e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,1.350651e+08,0.00,0.0,1.193409e+10,2.235586e+10,9.088775e+08,8.602612e+09,1.543971e+10,69.063375
20344,B5919,31.07.20,4.594674e+09,1.647843e+09,4.975718e+08,2.130367e+07,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,0.000000e+00,0.00,0.0,6.761393e+09,2.528626e+10,2.946595e+09,7.786492e+08,1.495347e+10,59.136725
20345,B5919,31.08.18,6.296507e+09,1.425508e+08,1.850541e+08,0.000000e+00,0.000000e+00,0.0,0.0,0.0,...,0.000000e+00,2.274267e+07,0.00,0.0,6.624111e+09,7.129747e+09,1.139464e+09,8.444035e+09,1.420895e+10,199.291059


In [70]:
dfX_01['REPORTINGDATE'] = pd.to_datetime(dfX_01['REPORTINGDATE'])

C:\Users\jqb_p\AppData\Local\Temp\ipykernel_1192\861186548.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dfX_01['REPORTINGDATE'] = pd.to_datetime(dfX_01['REPORTINGDATE'])


In [72]:
df_monthly['BSH_REPORTINGDATE'] = pd.to_datetime(df_monthly['BSH_REPORTINGDATE'])

C:\Users\jqb_p\AppData\Local\Temp\ipykernel_1192\87189534.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_monthly['BSH_REPORTINGDATE'] = pd.to_datetime(df_monthly['BSH_REPORTINGDATE'])


In [73]:
df_weekly_test = dfX_01[dfX_01['REPORTINGDATE'].dt.year>=2019]
df_weekly_train = dfX_01[dfX_01['REPORTINGDATE'].dt.year<2019]

df_monthly_test = df_monthly[df_monthly['BSH_REPORTINGDATE'].dt.year>=2019]
df_monthly_train = df_monthly[df_monthly['BSH_REPORTINGDATE'].dt.year<2019]

In [74]:
df_weekly_test = df_weekly_test.set_index('REPORTINGDATE')
df_weekly_train = df_weekly_train.set_index('REPORTINGDATE')

df_monthly_test = df_monthly_test.set_index('BSH_REPORTINGDATE')
df_monthly_train = df_monthly_train.set_index('BSH_REPORTINGDATE')

In [75]:
df_weekly_train

DESC_NO,INSTITUTIONCODE,01_CURR_ACC,02_TIME_DEPOSIT,03_SAVINGS,04_OTHER_DEPOSITS,05_BANKS_DEPOSITS,06_BORROWING_FROM_PUBLIC,07_INTERBANKS_LOAN_PAYABLE,08_CHEQUES_ISSUED,09_PAY_ORDERS,...,23_OTHER_GOV_SECURITIES,24_FOREIGN_CURRENCY,25_COMMERCIAL_BILLS,26_PROMISSORY_NOTES,XX_CUSTOMER_DEPOSITS,XX_TOTAL_LIQUID_LIAB,XX_BOT_BALANCE,XX_BAL_IN_OTHER_BANKS,XX_TOTAL_LIQUID_ASSET,XX_MLA
REPORTINGDATE,,,,,,,,,,,,,,,,,,,,,
2016-01-01,B5014,7.398879e+10,5.628409e+10,9.143355e+10,2.559228e+08,0.0,0.0,0.0,402901067.9,2.570451e+09,...,0.000000e+00,2.878479e+10,0.04,0.0,2.219623e+11,2.468379e+11,7.136215e+10,5.333950e+10,1.981265e+11,80.265835
2013-01-02,B5014,2.806429e+10,2.871621e+10,4.605558e+10,6.198604e+08,0.0,0.0,0.0,277307475.2,3.300202e+09,...,0.000000e+00,1.079983e+10,-0.01,0.0,1.034559e+11,1.168572e+11,2.457452e+10,1.490560e+10,1.071388e+11,91.683533
2013-01-03,B5014,6.784919e+10,2.463718e+10,4.675235e+10,6.241014e+08,0.0,0.0,0.0,308284811.6,3.295242e+09,...,0.000000e+00,1.396014e+10,-0.01,0.0,1.398628e+11,1.530413e+11,2.630510e+10,1.944015e+10,1.135819e+11,74.216474
2011-01-04,B5014,4.649237e+10,1.070078e+10,2.437827e+10,4.687598e+08,0.0,0.0,0.0,206865862.3,1.069560e+07,...,0.000000e+00,4.435655e+09,388791.09,0.0,8.204018e+10,1.035198e+11,2.990039e+10,1.420729e+10,9.979143e+10,96.398409
2016-01-04,B5014,1.016520e+11,5.677656e+10,9.129218e+10,4.458724e+09,0.0,0.0,0.0,299640414.3,3.261496e+08,...,6.726076e+09,1.177462e+10,0.04,0.0,2.541795e+11,2.701993e+11,5.412130e+10,3.936837e+10,1.800682e+11,66.642711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-28,B5919,5.080250e+09,1.458475e+08,5.582088e+08,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,2.041234e+08,0.00,0.0,5.784306e+09,1.058096e+10,8.070156e+08,4.126672e+09,1.150629e+10,108.745268
2018-06-29,B5919,4.285567e+09,4.256767e+07,1.314463e+08,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,8.579573e+07,0.00,0.0,4.459580e+09,5.524117e+09,8.633692e+08,3.446658e+09,9.124847e+09,165.182002
2018-03-30,B5919,3.523834e+08,2.010000e+07,6.748478e+07,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00,...,0.000000e+00,6.995724e+07,0.00,0.0,4.399682e+08,7.056128e+08,1.011201e+09,1.149706e+09,5.577677e+09,790.472656


In [76]:
df_monthly_test

,INSTITUTIONCODE,F001_ASSET_CASH,F002_ASSET_BAL_BOT,F003_ASSET_BAL_BOT_SMR,F004_ASSET_BAL_BOT_CURRENT_ACCOUNT,F005_ASSET_BAL_BOT_OTHERS,F006_ASSET_BAL_OTHER_BANKS,F007_ASSET_BAL_OTHER_BANKS_TZ,F008_ASSET_BAL_OTHER_BANKS_ABROAD,F009_ASSET_BAL_OTHER_BANKS_ABROAD_FIXED_INV,...,EWAQ_Capital,EWAQ_NPL,EWAQ_GrossLoans,EWAQ_LargeExposures,EWAQ_NPLsNetOfProvisions,EWAQ_NPLsNetOfProvisions2CoreCapital,EWAQ_NPLs2GrossLoans,EWAQ_AssetsQualityRating,EWAQ_Loans,MLA_CLASS
BSH_REPORTINGDATE,,,,,,,,,,,,,,,,,,,,,
2019-01-31,B5412,2.066706e+09,1.800000e+11,0.0,1.600000e+11,2.000000e+10,3.950000e+11,1.258310e+10,3.830000e+11,1.950000e+11,...,8.671695e+10,0.0,1.730000e+11,5.040000e+11,-221729180.0,-0.0026,0.0,2.0,NaN,high
2019-02-28,B5412,4.094379e+09,1.280000e+11,0.0,1.280000e+11,0.000000e+00,3.790000e+11,2.239447e+10,3.560000e+11,1.850000e+11,...,8.582212e+10,0.0,2.110000e+11,2.830000e+11,-177404829.0,-0.0021,0.0,1.0,NaN,high
2019-03-31,B5412,5.656643e+09,8.541262e+10,0.0,8.541262e+10,0.000000e+00,3.260000e+11,2.396660e+09,3.240000e+11,1.160000e+11,...,8.677562e+10,0.0,2.520000e+11,6.730000e+11,-156575960.0,-0.0018,0.0,2.0,NaN,high
2019-04-30,B5412,7.018719e+09,8.234253e+10,0.0,8.234253e+10,0.000000e+00,3.500000e+11,1.664563e+10,3.330000e+11,1.710000e+11,...,8.658118e+10,0.0,2.900000e+11,3.700000e+11,-272711100.0,-0.0031,0.0,2.0,NaN,high
2019-05-31,B5412,4.304272e+09,9.875559e+10,0.0,9.875559e+10,0.000000e+00,2.640000e+11,7.772844e+09,2.570000e+11,1.560000e+11,...,8.605636e+10,0.0,3.040000e+11,3.930000e+11,-307247462.0,-0.0036,0.0,2.0,NaN,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,B5120,6.934755e+08,1.703142e+09,0.0,1.703142e+09,0.000000e+00,2.742275e+08,2.746237e+08,0.000000e+00,0.000000e+00,...,9.314589e+10,0.0,2.274679e+10,1.653192e+10,-5807292.0,-0.0001,0.0,1.0,NaN,high
2021-09-30,B5120,3.887219e+08,1.388285e+09,0.0,1.388285e+09,0.000000e+00,8.007882e+08,8.035486e+08,0.000000e+00,0.000000e+00,...,9.337502e+10,0.0,2.301912e+10,1.657398e+10,-5749760.0,-0.0001,0.0,1.0,NaN,high
2021-10-31,B5120,4.772036e+08,7.915250e+08,0.0,7.915250e+08,0.000000e+00,2.992667e+08,3.003208e+08,0.000000e+00,0.000000e+00,...,9.359472e+10,0.0,2.318983e+10,3.300312e+10,-3640727.0,0.0000,0.0,1.0,NaN,high


# Preprocessing of monthly (df_monthly_xxx) and weekly (df_weekly_xxx) datasets finished, let's feed them into TFT

In [77]:
import numpy as np
import pandas as pd
from TFT import TFT

In [78]:
tft = TFT(cache_data=False)

In [79]:
df_y_test=df_weekly_test.pop('XX_MLA')
df_y_train=df_weekly_train.pop('XX_MLA')

In [80]:
df_y_train=pd.DataFrame(df_y_train)
df_y_test=pd.DataFrame(df_y_test)

In [81]:
tft.fit(
    X={"W": df_weekly_train, "ME": df_monthly_train},
    y={"W": df_y_train}
)

Processing `X` and `y` data.
  working out the frequencies...


NameError: name 'y_freq' is not defined